In [1]:
import pandas as pd
import numpy as np
from torch import device, cuda, Tensor, tensor, zeros, save, load
from sentence_transformers import SentenceTransformer, util

In [2]:
from tools import tools

In [3]:
model_name = 'all-mpnet-base-v2'

In [4]:
device = device('cuda' if cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
reports_file_path = 'data/br/firefox/firefox.csv'
relations_file_path = 'data/br/firefox/firefox_pairs.csv'

reports_firefox = pd.read_csv(reports_file_path, index_col='bug_id')
relations_firefox = pd.read_csv(relations_file_path)

In [6]:
embeddings_file_path = 'firefox_mpnet_embeddings.csv'
embeddings_firefox = pd.read_csv(
        embeddings_file_path, 
        index_col='bug_id'        
    )

In [7]:
def string_to_tensor(input_data):
    if input_data == 'nan' or input_data == None: #or (isinstance(input_data, float) and np.isnan(input_data)):
        # Return a default tensor for NaN values
        # Adjust the size of the tensor as needed
        return zeros((1,))  # Example: a tensor of size 1 with zeros
    elif isinstance(input_data, str):
        array = np.fromstring(input_data[1:-1], sep=' ')
        return tensor(array)
    elif isinstance(input_data, (int, float, list, np.ndarray)):
        return tensor(input_data)
    else:
        raise TypeError(f"Unsupported type for conversion to tensor: {type(input_data)}")

In [8]:
embeddings_firefox[model_name] = embeddings_firefox[model_name].apply(
    lambda x: tensor(
        string_to_tensor(x)
    )
)

C:\Users\lpros\AppData\Local\Temp\ipykernel_18892\340503249.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lambda x: tensor(


In [9]:
i = 0
duplicate_ids = []
bug_id = 0
count = 0
for index, row in relations_firefox.iterrows():

    bug_id = row[0]

    if ( not pd.isna(row[1])):

        if (count == 9):
            duplicate_ids = [
                int(x) for x in row[1].split(";") if int(x) in reports_firefox.index
            ]

            duplicate_ids = set(duplicate_ids)
            break
        else:
            count+=1

print(bug_id)
    
        
duplicate_ids

127244


C:\Users\lpros\AppData\Local\Temp\ipykernel_18892\3391872290.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bug_id = row[0]
C:\Users\lpros\AppData\Local\Temp\ipykernel_18892\3391872290.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ( not pd.isna(row[1])):
C:\Users\lpros\AppData\Local\Temp\ipykernel_18892\3391872290.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  int(x) for x in row[1].split(";") if int(x) in report

{216966, 257364, 316329, 344035}

In [10]:
prompt = embeddings_firefox.loc[bug_id]
prompt

all-mpnet-base-v2    [tensor(0.0002, dtype=torch.float64), tensor(-...
Name: 127244, dtype: object

In [11]:
type(embeddings_firefox.iloc[0][model_name])

torch.Tensor

In [12]:
tensor_dict = embeddings_firefox[model_name].to_dict()

In [13]:
save(tensor_dict, 'firefox_embeddings_torch_file.pt')

In [14]:
load_tensor_dict = load('firefox_embeddings_torch_file.pt')

In [15]:
type(embeddings_firefox.iloc[0][model_name])
#prompt[model_name] = string_to_tensor(prompt[model_name])
type(prompt[model_name])

torch.Tensor

In [16]:
tools.rr_k(prompt=prompt, df=embeddings_firefox, model_name=model_name, duplicate_ids=duplicate_ids, k=10)

0.75

In [17]:
tools.rr_k_dict(prompt=prompt[model_name],tensor_dict=load_tensor_dict, model_name=model_name, duplicate_ids=duplicate_ids, k=10 )

0.75